In [7]:
# Gerekli kütüphaneleri import edelim
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import UnstructuredPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFium2Loader

In [8]:
# .env dosyasındaki environment variable'ları yükle
load_dotenv()

True

In [9]:
# Google API anahtarını kontrol et
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("GOOGLE_API_KEY environment variable not set!")

In [10]:
# Veri klasörünün yolu
DATA_PATH = '../data/' # PDF'lerin bulunduğu klasör

# Vektör veritabanını kaydedeceğimiz yer
DB_FAISS_PATH = 'vectorstore/db_faiss'

In [11]:
# 1. Adım: PDF'leri Yükleme
# DirectoryLoader, belirtilen klasördeki tüm PDF'leri yükler
# Bu, "unstructured" kütüphanesinin karmaşıklığını tamamen ortadan kaldırır.
loader = DirectoryLoader(DATA_PATH,
                         glob='**/*.pdf',
                         loader_cls=PyPDFium2Loader,
                         show_progress=True,
                         use_multithreading=True) # Hız için bunu açık bırakabiliriz
print("Hızlı ve güvenli modda PDF'ler yükleniyor...")
documents = loader.load()
print(f"{len(documents)} adet doküman (sayfa) yüklendi.")


Hızlı ve güvenli modda PDF'ler yükleniyor...


c:\Users\themu\financial-analyst-chatbot\venv\Lib\site-packages\pypdfium2\_helpers\textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")
100%|██████████| 30/30 [00:47<00:00,  1.60s/it]

5787 adet doküman (sayfa) yüklendi.


In [12]:
# 2. Adım: Dokümanları Parçalara Ayırma (Chunking)
# RAG mimarisinde, uzun dokümanları daha küçük ve yönetilebilir parçalara böleriz.
# Bu, arama doğruluğunu artırır.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)
print(f"Dokümanlar {len(docs)} adet parçaya (chunk) bölündü.")

Dokümanlar 12403 adet parçaya (chunk) bölündü.


In [13]:
# 3. Adım: Gömme (Embedding) Modelini Oluşturma
# Her bir metin parçasını sayısal bir vektöre dönüştürmek için Gemini'nin embedding modelini kullanacağız.
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [14]:
# 4. Adım: Vektör Veritabanı Oluşturma ve Kaydetme
# Tüm metin parçalarını ve onların vektör karşılıklarını FAISS veritabanına kaydediyoruz.
# Bu işlem, veri setinizin büyüklüğüne göre biraz zaman alabilir.
db = FAISS.from_documents(docs, embeddings)
db.save_local(DB_FAISS_PATH)

print("Vektör veritabanı başarıyla oluşturuldu ve 'vectorstore/db_faiss' klasörüne kaydedildi.")

Vektör veritabanı başarıyla oluşturuldu ve 'vectorstore/db_faiss' klasörüne kaydedildi.
